In [14]:
# ==============================================================================
# NOTEBOOK DE TESTES METAMÓRFICOS - NÍVEL 1: ANÁLISE DE SENSIBILIDADE
# ==============================================================================
# Objetivo: Avaliar a estabilidade e a sensibilidade do sistema a perturbações,
# ausências e valores extremos nos dados de entrada (caixa-preta estrita).
# ------------------------------------------------------------------------------

# 📦 Imports
import pandas as pd
import numpy as np
import joblib
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from pathlib import Path

# ==============================================================================
# 📁 ETAPA 1: SETUP DO AMBIENTE
# ==============================================================================

ethanol_path = Path(
    r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/data/[Etanol] Trajeto Casa-Escola-UFRN/trackLog-2023-Feb-13_06-38-49_seg.csv"
)
gasoline_path = Path(
    r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/data/[Gasolina] Trajeto Casa-Escola-UFRN/trackLog-2022-Dec-01_06-43-57_qui.csv"
)

df_ethanol = pd.read_csv(ethanol_path)
df_gasoline = pd.read_csv(gasoline_path)

print("Arquivos de dados carregados com sucesso.")

# ==============================================================================
# 🔧 PREPARAÇÃO DOS DADOS
# ==============================================================================

def preparar_dados(df):
    df = df.copy()
    df.columns = df.columns.str.strip()

    col_map = {
        "Speed (OBD)(km/h)": "Speed(OBD)(km/h)",
        "Latitude ": "Latitude",
        "Longitude ": "Longitude"
    }
    df.rename(columns={k: v for k, v in col_map.items() if k in df.columns}, inplace=True)

    if "Speed(OBD)(km/h)" in df.columns:
        speed_ms = df["Speed(OBD)(km/h)"].fillna(0).values * 1000 / 3600
        df["Acceleration"] = np.diff(speed_ms, prepend=speed_ms[0])

    return df

df_ethanol = preparar_dados(df_ethanol)
df_gasoline = preparar_dados(df_gasoline)

# ==============================================================================
# 🤖 CARREGAMENTO DOS MODELOS
# ==============================================================================

modelos = {
    "ethanol": {
        "afr": joblib.load(
            Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/models/LGBMRegressor_ethanol_afr.pkl")
        ),
        "maf": joblib.load(
            Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/models/XGBRegressor_ethanol_maf.pkl")
        ),
    },
    "gasoline": {
        "afr": joblib.load(
            Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/models/LGBMRegressor_gasoline_afr.pkl")
        ),
        "maf": joblib.load(
            Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/models/XGBRegressor_gasoline_maf.pkl")
        ),
    }
}

print("Modelos carregados com sucesso.")

# ==============================================================================
# 🧮 FUNÇÕES AUXILIARES
# ==============================================================================

def compute_co2(afr, maf, fuel):
    afr = np.asarray(afr)
    maf = np.asarray(maf)
    carbon_fraction = {"ethanol": 0.5217, "gasoline": 0.8571}
    fuel_rate = np.divide(maf, afr, out=np.zeros_like(maf), where=afr != 0)
    return fuel_rate * carbon_fraction[fuel] * (44.0 / 12.0)

def encontrar_coluna(df, nomes):
    for n in nomes:
        if n in df.columns:
            return df[n]
    return None

def pearson_safe(x, y):
    x, y = np.asarray(x), np.asarray(y)
    mask = ~np.isnan(x) & ~np.isnan(y)
    if mask.sum() < 2:
        return np.nan
    return pearsonr(x[mask], y[mask])[0]

# ==============================================================================
# 🔬 TESTE METAMÓRFICO NUMÉRICO
# ==============================================================================

def executar_teste_plotly(df_original, df_modificado, modelo_afr, modelo_maf, fuel, nome_ct):
    features = ["Latitude", "Longitude", "Speed(OBD)(km/h)", "Acceleration"]

    for f in features:
        if f not in df_original.columns:
            df_original[f] = 0
        if f not in df_modificado.columns:
            df_modificado[f] = 0

    afr_real = encontrar_coluna(
        df_original,
        ["AirFuelRatio(Commanded)(:1)", "Air Fuel Ratio(Commanded)(:1)"]
    )
    maf_real = encontrar_coluna(
        df_original,
        ["MassAirFlowRate(g/s)", "Mass Air Flow Rate(g/s)"]
    )

    co2_real = (
        compute_co2(afr_real, maf_real, fuel)
        if afr_real is not None and maf_real is not None
        else None
    )

    co2_orig = compute_co2(
        modelo_afr.predict(df_original[features]),
        modelo_maf.predict(df_original[features]),
        fuel
    )

    co2_mod = compute_co2(
        modelo_afr.predict(df_modificado[features]),
        modelo_maf.predict(df_modificado[features]),
        fuel
    )

    rmse = mean_squared_error(co2_orig, co2_mod, squared=False)
    pearson_rm = pearson_safe(co2_orig, co2_mod)
    pearson_real = pearson_safe(co2_real, co2_mod)

    # ======================================================
    # 📢 BLOCO INFORMATIVO (ANTES DO GRÁFICO)
    # ======================================================
    print("\n" + "=" * 80)
    print(f"🧪 {nome_ct} ({fuel.upper()})")
    print("Tipo de Teste: Metamórfico – Análise de Sensibilidade")
    print(f"RMSE Orig×RM = {rmse:.4f}")
    print(f"Pearson Orig×RM = {pearson_rm:.3f}")
    print(f"Pearson Real×RM = {pearson_real:.3f}")
    print("=" * 80)

    # ======================================================
    # 📈 GRÁFICO
    # ======================================================
    fig = go.Figure()

    if co2_real is not None:
        fig.add_trace(go.Scatter(
            y=co2_real[:200],
            name="Real",
            line=dict(dash="dot", color="gray")
        ))

    fig.add_trace(go.Scatter(
        y=co2_orig[:200],
        name="Predição Original",
        line=dict(dash="dash", color="blue")
    ))

    fig.add_trace(go.Scatter(
        y=co2_mod[:200],
        name="Predição com RM",
        line=dict(color="red")
    ))

    fig.update_layout(
        title=dict(
            text=(
                f"{nome_ct} – Comparação Metamórfica ({fuel.upper()})<br>"
                f"RMSE={rmse:.3f} | Pearson Orig×RM={pearson_rm:.3f}"
            ),
            x=0.5
        ),
        xaxis_title="Amostras",
        yaxis_title="Variável de Saída (CO₂)",
        template="plotly_white"
    )

    fig.show()


# ==============================================================================
# 🧪 TESTE DE ROBUSTEZ (NULL)
# ==============================================================================

def executar_teste_null(df_original, df_modificado, modelo_afr, modelo_maf, fuel, nome_ct):
    features = ["Latitude", "Longitude", "Speed(OBD)(km/h)", "Acceleration"]

    for f in features:
        if f not in df_original.columns:
            df_original[f] = 0
        if f not in df_modificado.columns:
            df_modificado[f] = 0

    afr_real = encontrar_coluna(
        df_original,
        ["AirFuelRatio(Commanded)(:1)", "Air Fuel Ratio(Commanded)(:1)"]
    )
    maf_real = encontrar_coluna(
        df_original,
        ["MassAirFlowRate(g/s)", "Mass Air Flow Rate(g/s)"]
    )

    co2_real = (
        compute_co2(afr_real, maf_real, fuel)
        if afr_real is not None and maf_real is not None
        else None
    )

    co2_orig = compute_co2(
        modelo_afr.predict(df_original[features]),
        modelo_maf.predict(df_original[features]),
        fuel
    )

    co2_null = compute_co2(
        modelo_afr.predict(df_modificado[features]),
        modelo_maf.predict(df_modificado[features]),
        fuel
    )

    rmse = mean_squared_error(co2_orig, co2_null, squared=False)
    pearson_null = pearson_safe(co2_orig, co2_null)
    pearson_real = pearson_safe(co2_real, co2_null)

    status = "❌ FALHA – NULL processado sem rejeição explícita"

    # ======================================================
    # 📢 BLOCO INFORMATIVO (ANTES DO GRÁFICO)
    # ======================================================
    print("\n" + "=" * 80)
    print(f"🧪 {nome_ct} ({fuel.upper()})")
    print("Tipo de Teste: Robustez – Valor NULL")
    print(f"Status do Teste: {status}")
    print(f"RMSE Orig×NULL = {rmse:.4f}")
    print(f"Pearson Orig×NULL = {pearson_null:.3f}")
    print(f"Pearson Real×NULL = {pearson_real:.3f}")
    print("=" * 80)

    # ======================================================
    # 📈 GRÁFICO
    # ======================================================
    fig = go.Figure()

    if co2_real is not None:
        fig.add_trace(go.Scatter(
            y=co2_real[:200],
            name="Real",
            line=dict(dash="dot", color="gray")
        ))

    fig.add_trace(go.Scatter(
        y=co2_orig[:200],
        name="Predição Original",
        line=dict(dash="dash", color="blue")
    ))

    fig.add_trace(go.Scatter(
        y=co2_null[:200],
        name="Predição com NULL",
        line=dict(color="red")
    ))

    fig.update_layout(
        title=dict(
            text=(
                f"{nome_ct} – Comparação com NULL ({fuel.upper()})<br>"
                f"{status}<br>"
                f"RMSE={rmse:.3f} | Pearson Null={pearson_null:.3f} | Pearson Real×NULL={pearson_real:.3f}"
            ),
            x=0.5
        ),
        xaxis_title="Amostras",
        yaxis_title="Variável de Saída (CO₂)",
        template="plotly_white"
    )

    fig.show()


# ==============================================================================
# 🔁 CASOS DE TESTE – NÍVEL 1
# ==============================================================================

testes_exclusivos = {
    "CT_N1_001 – Zerar A": lambda df: df.assign(**{"Speed(OBD)(km/h)": 0.0}),
    "CT_N1_002 – Zerar B": lambda df: df.assign(Acceleration=0.0),
    "CT_N1_003 – Zerar C": lambda df: df.assign(Latitude=0.0),
    "CT_N1_004 – Zerar D": lambda df: df.assign(Longitude=0.0),
}

testes_null = {
    "CT_N1_005 – A = null": lambda df: df.assign(**{"Speed(OBD)(km/h)": np.nan}),
    "CT_N1_006 – B = null": lambda df: df.assign(Acceleration=np.nan),
    "CT_N1_007 – C = null": lambda df: df.assign(Latitude=np.nan),
    "CT_N1_008 – D = null": lambda df: df.assign(Longitude=np.nan),
}

# ==============================================================================
# ▶️ EXECUÇÃO
# ==============================================================================

print("\n=== TESTES METAMÓRFICOS NUMÉRICOS ===")
for nome, t in testes_exclusivos.items():
    for fuel, df in [("ethanol", df_ethanol), ("gasoline", df_gasoline)]:
        executar_teste_plotly(df, t(df.copy()), modelos[fuel]["afr"], modelos[fuel]["maf"], fuel, nome)

print("\n=== TESTES DE ROBUSTEZ (NULL) ===")
for nome, t in testes_null.items():
    for fuel, df in [("ethanol", df_ethanol), ("gasoline", df_gasoline)]:
        executar_teste_null(df, t(df.copy()), modelos[fuel]["afr"], modelos[fuel]["maf"], fuel, nome)


Arquivos de dados carregados com sucesso.
Modelos carregados com sucesso.

=== TESTES METAMÓRFICOS NUMÉRICOS ===

🧪 CT_N1_001 – Zerar A (ETHANOL)
Tipo de Teste: Metamórfico – Análise de Sensibilidade
RMSE Orig×RM = 0.4900
Pearson Orig×RM = 0.395
Pearson Real×RM = 0.406



🧪 CT_N1_001 – Zerar A (GASOLINE)
Tipo de Teste: Metamórfico – Análise de Sensibilidade
RMSE Orig×RM = 0.8179
Pearson Orig×RM = 0.639
Pearson Real×RM = 0.537



🧪 CT_N1_002 – Zerar B (ETHANOL)
Tipo de Teste: Metamórfico – Análise de Sensibilidade
RMSE Orig×RM = 0.1847
Pearson Orig×RM = 0.919
Pearson Real×RM = 0.779



🧪 CT_N1_002 – Zerar B (GASOLINE)
Tipo de Teste: Metamórfico – Análise de Sensibilidade
RMSE Orig×RM = 0.2673
Pearson Orig×RM = 0.941
Pearson Real×RM = 0.733



🧪 CT_N1_003 – Zerar C (ETHANOL)
Tipo de Teste: Metamórfico – Análise de Sensibilidade
RMSE Orig×RM = 0.3941
Pearson Orig×RM = 0.480
Pearson Real×RM = 0.458


...


🧪 CT_N1_006 – B = null (GASOLINE)
Tipo de Teste: Robustez – Valor NULL
Status do Teste: ❌ FALHA – NULL processado sem rejeição explícita
RMSE Orig×NULL = 0.3328
Pearson Orig×NULL = 0.906
Pearson Real×NULL = 0.712



🧪 CT_N1_007 – C = null (ETHANOL)
Tipo de Teste: Robustez – Valor NULL
Status do Teste: ❌ FALHA – NULL processado sem rejeição explícita
RMSE Orig×NULL = 0.3941
Pearson Orig×NULL = 0.480
Pearson Real×NULL = 0.458



🧪 CT_N1_007 – C = null (GASOLINE)
Tipo de Teste: Robustez – Valor NULL
Status do Teste: ❌ FALHA – NULL processado sem rejeição explícita
RMSE Orig×NULL = 1.0747
Pearson Orig×NULL = 0.030
Pearson Real×NULL = 0.020



🧪 CT_N1_008 – D = null (ETHANOL)
Tipo de Teste: Robustez – Valor NULL
Status do Teste: ❌ FALHA – NULL processado sem rejeição explícita
RMSE Orig×NULL = 0.4241
Pearson Orig×NULL = 0.385
Pearson Real×NULL = 0.351



🧪 CT_N1_008 – D = null (GASOLINE)
Tipo de Teste: Robustez – Valor NULL
Status do Teste: ❌ FALHA – NULL processado sem rejeição explícita
RMSE Orig×NULL = 0.7443
Pearson Orig×NULL = 0.456
Pearson Real×NULL = 0.398
